In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

In [2]:
raw_csv_data: pd.DataFrame = pd.read_csv("data/condition/condition_1.csv") 
df: pd.DataFrame = raw_csv_data.copy() 

In [3]:
sizes_df: pd.DataFrame = df.groupby('date').count()
display(sizes_df)

,timestamp,activity
date,,
2003-05-07,720,720
2003-05-08,1440,1440
2003-05-09,1440,1440
2003-05-10,1440,1440
2003-05-11,1440,1440
2003-05-12,1440,1440
2003-05-13,1440,1440
2003-05-14,1440,1440
2003-05-15,1440,1440


In [4]:
def pathCondition(id: int) -> str:
    return f'data/condition/condition_{id}.csv'

def pathControl(id:int) -> str:
    return f'data/control/control_{id}.csv'

In [5]:
path: str
complete: list[pd.DataFrame] = []
not_complete: list[pd.DataFrame] = []
for i in range(1,24): 
    path = pathCondition(i)
    raw_csv_data = pd.read_csv(path) 
    df = raw_csv_data.copy() 
    sizes_df = df.groupby('date').count()
    sizes_df['path'] = path
    not_complete.append(sizes_df.loc[sizes_df['timestamp']!=1440])
    complete.append(sizes_df.loc[sizes_df['timestamp']==1440])
    
for i in range(1,33): 
    path = pathControl(i)
    raw_csv_data = pd.read_csv(path) 
    df = raw_csv_data.copy() 
    sizes_df = df.groupby('date').count()
    sizes_df['path'] = path
    not_complete.append(sizes_df.loc[sizes_df['timestamp']!=1440])
    complete.append(sizes_df.loc[sizes_df['timestamp']==1440])

complete_df: pd.DataFrame = pd.concat(complete)
not_complete_df: pd.DataFrame = pd.concat(not_complete)
print(f'Number of complete days: {len(complete_df)}')
print(f'Number of not complete days: {len(not_complete_df)}')

Number of complete days: 1029
Number of not complete days: 115


In [6]:
complete_df.head()

,timestamp,activity,path
date,,,
2003-05-08,1440,1440,data/condition/condition_1.csv
2003-05-09,1440,1440,data/condition/condition_1.csv
2003-05-10,1440,1440,data/condition/condition_1.csv
2003-05-11,1440,1440,data/condition/condition_1.csv
2003-05-12,1440,1440,data/condition/condition_1.csv


In [7]:
name: str
filtered_df: pd.DataFrame
time_series: list[pd.DataFrame] = []
for idx, row in complete_df.iterrows():
    name = row['path'].split('/')[-1][:-4]
    df = pd.read_csv(row['path'])
    filtered_df = df[df.date == idx].copy().set_index('date')
    filtered_df['owner'] = name
    time_series.append(filtered_df)

time_series_df = pd.concat(time_series, keys=range(len(time_series)), names=['id', 'date'])

In [8]:
time_series_df.head()

timestamp  activity        owner
id date                                                  
0  2003-05-08  2003-05-08 00:00:00         0  condition_1
   2003-05-08  2003-05-08 00:01:00         0  condition_1
   2003-05-08  2003-05-08 00:02:00         0  condition_1
   2003-05-08  2003-05-08 00:03:00         0  condition_1
   2003-05-08  2003-05-08 00:04:00         0  condition_1

In [9]:
print(f'Expected size: {len(complete_df)*1440}')
print(f'Size of time_series_df: {len(time_series_df)}')

Expected size: 1481760
Size of time_series_df: 1481760


In [10]:
time_series_df.to_csv('time_series.csv', index=True)